# Rules extractor

Use this notebook to extract low-level rules from the game (ie: create low level tiles).

In [ ]:
from pcgsepy.common.api_call import call_api, generate_json
from pcgsepy.common.vecs import Orientation, orientation_from_vec, Vec
from pcgsepy.structure import Block

In [ ]:
obs = call_api(jsons=[
    generate_json(method="Observer.ObserveBlocks")
    ])[0]

In [ ]:
grids = obs['result']['Grids']

In [ ]:
print(f'Found {len(grids)} grids of blocks:')
for i, grid in enumerate(grids):
    print(f"  {i+1}. Grid containing {len(grid['Blocks'])} blocks")
grid = int(input('Choose which grid to process (number): ')) - 1
assert grid > -1 and grid < len(grids), f'Invalid grid index: {grid}'
rule_name = input('Enter name of tile: ')

In [ ]:
grid = grids[grid]
blocks = []
for block in grid['Blocks']:
    block_type = block['DefinitionId']['Type']
    orientation_forward = orientation_from_vec(Vec.from_json(j=block['OrientationForward']))
    orientation_up = orientation_from_vec(Vec.from_json(j=block['OrientationUp']))
    position = block['Position']
    ds_block = Block(block_type=block_type,
                     orientation_forward=orientation_forward,
                     orientation_up=orientation_up)
    ds_block.position = Vec.from_json(j=position)
    blocks.append(ds_block)

In [ ]:
max_x, max_y, max_z = 0., 0., 0.

for block in blocks:
    x, y, z = block.position.as_tuple()
    if x > max_x:
        max_x = x
    if y > max_y:
        max_y = y
    if z > max_z:
        max_z = z

In [ ]:
min_x, min_y, min_z = max_x, max_y, max_z

for block in blocks:
    x, y, z = block.position.as_tuple()
    if x < min_x:
        min_x = x
    if y < min_y:
        min_y = y
    if z < min_z:
        min_z = z

In [ ]:
for block in blocks:
    block.position = block.position.sum(Vec.v3f(-min_x, -min_y, -min_z))

In [ ]:
max_x -= min_x
max_y -= min_y
max_z -= min_z

In [ ]:
def at_same_x(x, blocks):
    r = []
    for b in blocks:
        if b.position.x == x:
            r.append(b)
    return r


def at_same_y(y, blocks):
    r = []
    for b in blocks:
        if b.position.y == y:
            r.append(b)
    return r


def at_same_z(z, blocks):
    r = []
    for b in blocks:
        if b.position.z == z:
            r.append(b)
    return r

In [ ]:
ordered_blocks = []
x, y, z = 0., 0., 0.
while z <= max_z:
    bs1 = at_same_z(z, blocks)
    while y <= max_y:
        bs2 = at_same_y(y, bs1)
        while x <= max_x:
            b = at_same_x(x, bs2)
            if b:
                ordered_blocks.append(b[0])
            x += 0.5
        x = 0.
        y += 0.5
    x = 0.
    y = 0.
    z += 0.5

In [ ]:
orientations_str = {
    Orientation.FORWARD: 'F',
    Orientation.BACKWARD: 'B',
    Orientation.RIGHT: 'R',
    Orientation.LEFT: 'L',
    Orientation.UP: 'U',
    Orientation.DOWN: 'D',
}

In [ ]:
rule = ''

x, y, z = 0., 0., 0.
for block in ordered_blocks:
    if block.position.z != z:
        if block.position.z > z:
            dz = block.position.z - z
            rule += f'>({int(dz // 0.5)})'
            z = block.position.z
        else:
            dz = z - block.position.z
            rule += f'<({int(dz // 0.5)})'
            z = block.position.z
    if block.position.y != y:
        if block.position.y > y:
            dy = block.position.y - y
            rule += f'!({int(dy // 0.5)})'
            y = block.position.y
        else:
            dy = y - block.position.y
            rule += f'?({int(dy // 0.5)})'
            y = block.position.y
    if block.position.x != x:
        if block.position.x > x:
            dx = block.position.x - x
            rule += f'+({int(dx // 0.5)})'
            x = block.position.x
        else:
            dx = x - block.position.x
            rule += f'-({int(dx // 0.5)})'
            x = block.position.x
    of = orientations_str[orientation_from_vec(block.orientation_forward)]
    ou = orientations_str[orientation_from_vec(block.orientation_up)]
    rule += f'{block.block_type}({of},{ou})'

if x != 0.:
    if x > 0.:
        rule += f'-({int(x // 0.5)})'
    if x < 0.:
        rule += f'+({int(-x // 0.5)})'
if y != 0.:
    if y > 0.:
        rule += f'?({int(y // 0.5)})'
    if y < 0.:
        rule += f'!({int(-y // 0.5)})'
if z != 0.:
    if z > 0.:
        rule += f'<({int(z // 0.5)})'
    if z < 0.:
        rule += f'>({int(-z // 0.5)})'

In [ ]:
print(f'RULE: {rule_name}')
print(rule)